In [1]:
import gensim
import nltk
import pandas as pd

In [2]:
def lower(x):
    return x.lower()

In [8]:
df = pd.read_csv('news_data.csv')
df['news'] = df['news'].apply(lower)
news_list = df['news'].to_list()
news_corpus = ' '.join(news_list)

In [12]:
"_".join(['a','b'])

'a_b'

In [5]:
stop_words = nltk.corpus.stopwords.words('english')

In [6]:
tokens = nltk.tokenize.word_tokenize(news_corpus)

In [7]:
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()
#quadgrams = nltk.metrics.QuadgramAssocMeasures()

In [8]:
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(tokens)
#quadgramFinder = nltk.collocations.QuadgramCollocationFinder.from_words(tokens)

In [9]:
bigramFinder.apply_freq_filter(120)
trigramFinder.apply_freq_filter(30)
#quadgramFinder.apply_freq_filter(5)

In [10]:
# FILTER
# NOUN AND NOUN FOR BIGRAM
# NOUN, ANYTHING AND NOUN FOR TRIGRAM

#get english stopwords
import string
en_stopwords = set(stop_words)
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    for word in ngram:
        if word in en_stopwords or word.isspace() or word in (string.punctuation + '’') or word.isdigit():
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

#function to filter for trigrams
def rightTypesTri(ngram):
    for word in ngram:
        if word in en_stopwords or word.isspace() or word in (string.punctuation + '’') or word.isdigit():
            return False
    first_type = ('NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False


In [11]:
bigramTtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','t']).sort_values(by='t', ascending=False)
trigramTtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.student_t)), columns=['trigram','t']).sort_values(by='t', ascending=False)

In [12]:
#filters
filteredT_bi = bigramTtable[bigramTtable.bigram.map(lambda x: rightTypes(x))]
filteredT_tri = trigramTtable[trigramTtable.trigram.map(lambda x: rightTypesTri(x))]

In [13]:
filteredT_tri

,trigram,t
8,"(quarter, company, analysis)",41.363129
110,"(muktinath, bikas, bank)",14.931445
124,"(bonus, share, certificate)",14.247273
139,"(century, commercial, bank)",13.637708
147,"(cash, dividend, warrant)",13.341582
153,"(life, insurance, company)",13.226763
167,"(units, bonus, shares)",12.761222
205,"(jyoti, bikas, bank)",11.830677
214,"(units, bonus, share)",11.570803
225,"(units, ipo, shares)",11.440001


In [14]:
data_trigram = filteredT_tri[["trigram", "t"]].to_numpy()

def apply_trigram(list_tokens):
    for trigram, _ in data_trigram:
        while(trigram[0] in list_tokens[:-2]):
            index = list_tokens[:-2].index(trigram[0])
            if (trigram[1] == list_tokens[index+1]) and (trigram[2] == list_tokens[index+2]):
                list_tokens[index:index+3] = ['_'.join(trigram)]
            else:
                break
    return list_tokens
    

In [15]:
data_bigram = filteredT_bi[["bigram", "t"]].to_numpy()

def apply_bigram(list_tokens):
    for bigram, _ in data_bigram:
        while(bigram[0] in list_tokens[:-1]):
            index = list_tokens[:-1].index(bigram[0])
            if (bigram[1] == list_tokens[index+1]):
                list_tokens[index:index+2] = ['_'.join(bigram)]
            else:
                break
    
    return list_tokens

In [18]:
df.news = df.news.apply(nltk.tokenize.word_tokenize).apply(apply_trigram).apply(apply_bigram)

In [20]:
df.to_csv("processed_data.csv")

In [16]:
# big data need streamer (iterator)
class ProcessedCorpus:
    def __init__(self):
        self.data = processed_df.to_list()
    
    def __len__(self):
        return len(self.data)
    
    def __iter__(self):
        for sent in self.data:
            yield sent

In [17]:
dats = ProcessedCorpus()
for x in dats:
    print(x)
    break

['market', 'gains', 'minimum', 'points', 'after', 'losing', 'for', 'five', 'consecutive', 'days', ';', 'three', 'hydropower', 'companies', 'hit', 'the', 'circuit', 'today', ',']


In [18]:
FastText = gensim.models.FastText
model = FastText(size=100, window=5, min_count=10) 

In [19]:
model.build_vocab(sentences=ProcessedCorpus())

In [20]:
model.train(sentences=ProcessedCorpus(), total_examples=len(ProcessedCorpus()), epochs=10) 

In [21]:
model.most_similar('bank')

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('interbank', 0.9140117764472961),
 ('banker', 0.8847012519836426),
 ('ncc_bank', 0.8717657327651978),
 ('kumari_bank', 0.8648148775100708),
 ('mega_bank', 0.86028653383255),
 ('dev_bank', 0.8579362034797668),
 ('sunrise_bank', 0.8531262874603271),
 ('civil_bank', 0.8512514233589172),
 ('janata_bank', 0.8461079597473145),
 ('nmb_bank', 0.84540855884552)]

In [22]:
model.most_similar('nabil_bank')

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('nmb_bank', 0.9636020660400391),
 ('siddhartha_bank', 0.9598978757858276),
 ('civil_bank', 0.9579761028289795),
 ('laxmi_bank', 0.954574465751648),
 ('sanima_bank', 0.9541477560997009),
 ('ncc_bank', 0.9526793956756592),
 ('citizens_bank', 0.9514545798301697),
 ('everest_bank', 0.950179398059845),
 ('sunrise_bank', 0.949787437915802),
 ('kumari_bank', 0.9475828409194946)]

In [23]:
model.wv.save("model")

TypeError: 'numpy.ndarray' object is not callable